In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def block(inputs,filters=64,strides=1):
    """
    Block 1 of Resnet 18
    
    Args:
    inputs: A tensor of size [batch, height_in, width_in, channels]
    filters: The number of filters for the convolutions.
    strides: The block's stride.
    """
    shortcut = inputs
    
    inputs = tf.layers.conv2d(inputs, filters, kernel_size=[3,3], strides=[1,strides,strides,1], 
                              padding='SAME',use_bias=False, 
                              kernel_initializer= tf.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
    
    inputs = tf.nn.selu(inputs)
    
    inputs = tf.layers.conv2d(inputs, filters, kernel_size=[3,3], strides=[1,strides,strides,1], 
                              padding='SAME',use_bias=False, 
                              kernel_initializer= tf.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
    
    
    
    inputs += shortcut
    
    inputs = tf.nn.selu(inputs)
    
    return inputs
    
    

In [0]:
def _block_v1(inputs,filters=64,strides=1,downsample):
    """
    Block 1 of Resnet 18
    
    Args:
    inputs: A tensor of size [batch, height_in, width_in, channels]
    filters: The number of filters for the convolutions.
    strides: The block's stride.
    """
    shortcut = inputs
    
    if downsample is not None:
      shortcut = downsample(inputs)
    
    inputs = tf.layers.conv2d(inputs, filters, kernel_size=[3,3], strides=[1,strides,strides,1], 
                              padding='SAME',use_bias=False, 
                              kernel_initializer= tf.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
    
    inputs = tf.nn.selu(inputs)
    
    inputs = tf.layers.conv2d(inputs, filters, kernel_size=[3,3], strides=[1,strides,strides,1], 
                              padding='SAME',use_bias=False, 
                              kernel_initializer= tf.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
    
    
    
    inputs += shortcut
    
    inputs = tf.nn.selu(inputs)
    
    return inputs
    

In [0]:
def block_layer(inputs, filters, strides, name, training):
  
    def downsample(inputs):
      
  

In [0]:
def block_down_sample(inputs,filters,strides=1):
    """
    Block 2 of Resnet 18
    
    Args:
    inputs: A tensor of size [batch, height_in, width_in, channels]
    filters: The number of filters for the convolutions.
    strides: The block's stride.
    """
    shortcut = inputs
    
    inputs = tf.layers.conv2d(inputs, filters, kernel_size=[3,3], strides=[1,strides,strides,1], 
                              padding='SAME',use_bias=False, 
                              kernel_initializer= tf.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
    
    inputs = tf.nn.selu(inputs)
    
    inputs = tf.layers.conv2d(inputs, filters, kernel_size=[3,3], strides=[1,2,2,1], 
                              padding='SAME',use_bias=False, 
                              kernel_initializer= tf.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
    
    inputs += shortcut
    
    inputs = tf.nn.selu(inputs)
    
    return inputs

In [0]:
class Model(object):
  
  def __init__(self, num_classes):
    
    """
    Args:
    
      num_classes: The number of classes used as labels.
    """
    
    self.num_classes = num_classes
    
    def __call__(self, inputs):
      
      """
      Add operations to classify a batch of input images.
      
      Args:
      inputs: A Tensor representing a batch of input images.
      
      Returns:
      A logits Tensor with shape [<batch_size>, self.num_classes].
      """
      
      
      inputs = tf.layers.conv2d(inputs, 64, kernel_size=[7,7], strides=[1,2,2,1], 
                              padding='SAME',use_bias=False, 
                              kernel_initializer= tf.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
      
      inputs = tf.nn.selu(inputs)
      
      inputs = tf.layers.max_pooling2d( inputs=inputs, pool_size=[3,3], strides=[1,2,2,1], padding='SAME')
      
      inputs = tf.identity(inputs)
      
      inputs = block(inputs)
      inputs = block(inputs)
      
      for i in range(3):
        inputs = block_down_sample(inputs, (i+7)**2)
        inputs = block(inputs,(i+7)**2)
        
      #Used Reduce_mean instead of AvgPooling2D as it performs better  
      axes = [1, 2]
      inputs = tf.reduce_mean(inputs, axes, keepdims=True)
      inputs = tf.identity(inputs, 'final_reduce_mean')

      inputs = tf.squeeze(inputs, axes)
      inputs = tf.layers.dense(inputs=inputs, units=self.num_classes)
      inputs = tf.identity(inputs, 'final_dense')
      return inputs
